# 464. Can I Win

## Topic Alignment
- Memoized DFS mirrors game-theoretic planning for resource allocation where state reuse avoids exponential blow-up.

## Metadata 摘要
- Source: https://leetcode.com/problems/can-i-win/
- Tags: Game Theory, DFS, Memoization, Bitmask
- Difficulty: Medium
- Priority: High

## Problem Statement 原题描述
Two players alternately choose integers from 1 to maxChoosableInteger without reuse. The running total increases by the chosen value, and the player whose move reaches or exceeds desiredTotal wins. Determine if the first player can force a win with optimal play.

## Progressive Hints
- Hint 1: If the sum of all numbers is less than desiredTotal, the first player cannot win.
- Hint 2: Use bitmasks to represent which numbers are already taken.
- Hint 3: A state is winning if any available move leads the opponent to a losing state.

## Solution Overview
Define dfs(mask, remaining) which returns True if the current player can force a win with the remaining total and already-used numbers encoded in mask. Iterate through numbers 1..max; if a number is unused and reaches remaining or leads to opponent failure, current state wins. Memoize results by mask and remaining to avoid recomputation.

## Detailed Explanation
1. Handle trivial cases: desiredTotal <= 0 => True; if total sum of range is less than desiredTotal => False.
2. Use functools.lru_cache to memoize dfs(mask, remaining).
3. For each number i from 1..maxChoosableInteger, if unused (`mask & bit == 0`), check two winning conditions:
   - i >= remaining (current player wins immediately).
   - dfs(mask | bit, remaining - i) returns False (opponent cannot win).
4. If no winning move exists, state is losing. Complexity is limited by 2^max states, with max <= 20.

## Complexity Trade-off Table
| Approach | Time | Space | Notes |
| --- | --- | --- | --- |
| DFS + memo (bitmask) | O(2^{max} * max) | O(2^{max}) | Practical for max ≤ 20 |
| Plain minimax without memo | Exponential | Exponential | Recomputes identical states repeatedly |

In [ ]:
from functools import lru_cache

class Solution:
    def canIWin(self, maxChoosableInteger: int, desiredTotal: int) -> bool:
        if desiredTotal <= 0:
            return True
        total_sum = maxChoosableInteger * (maxChoosableInteger + 1) // 2
        if total_sum < desiredTotal:
            return False
        @lru_cache(None)
        def dfs(mask: int, remaining: int) -> bool:
            for i in range(1, maxChoosableInteger + 1):
                bit = 1 << (i - 1)
                if mask & bit:
                    continue
                if i >= remaining:
                    return True
                if not dfs(mask | bit, remaining - i):
                    return True
            return False
        return dfs(0, desiredTotal)

In [ ]:
tests = [
    ((10, 11), False),
    ((10, 0), True),
    ((5, 6), True),
    ((20, 210), False)
]
solver = Solution()
for params, expected in tests:
    assert solver.canIWin(*params) == expected
print('All tests passed.')

## Complexity Analysis
- Time: O(2^{max} * max). Many states prune early due to quick wins.
- Space: O(2^{max}) memo entries plus recursion depth up to max.

## Edge Cases & Pitfalls
- Always check if total_sum < desiredTotal before recursing; otherwise you may waste time.
- Use bitmask for memo key; lists/tuples cause large memory overhead.
- Python recursion depth easily handles <= 20 moves.

## Follow-up Variants
- Alter win condition to exact match (no overshoot) and adapt base cases.
- Introduce penalties or rewards per move and compute optimal score difference.
- Extend to three players by adding turn index into the memo state.

## Takeaways
- Memoization turns exponential minimax into manageable complexity.
- Bitmask state encoding is a standard trick for small maxChoosableInteger.
- Early termination on guaranteed win/lose conditions saves time.

## Similar Problems
| Problem ID | Problem Title | Technique |
| --- | --- | --- |
| LC 486 | Predict the Winner | Memoized minimax |
| LC 877 | Stone Game | Game DP / memoization |
| LC 698 | Partition to K Equal Sum Subsets | DFS + memo on bitmask |